In [1]:

# Large amount of credit goes to:
# https://github.com/keras-team/keras-contrib/blob/master/examples/improved_wgan.py
# which I've used as a reference for this implementation

from __future__ import print_function, division

from keras.datasets import cifar10
from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from functools import partial
from PIL import Image
import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np
import os
class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

class WGANGP():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        optimizer = RMSprop(lr=0.00005)

        # Build the generator and critic
        self.generator = self.build_generator()
        self.critic = self.build_critic()

        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------

        # Freeze generator's layers while training critic
        self.generator.trainable = False

        # Image input (real sample)
        real_img = Input(shape=self.img_shape)

        # Noise input
        z_disc = Input(shape=(self.latent_dim,))
        # Generate image based of noise (fake sample)
        fake_img = self.generator(z_disc)

        # Discriminator determines validity of the real and fake images
        fake = self.critic(fake_img)
        valid = self.critic(real_img)

        # Construct weighted average between real and fake images
        interpolated_img = RandomWeightedAverage()([real_img, fake_img])
        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated_img)

        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss,
                          averaged_samples=interpolated_img)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic_model = Model(inputs=[real_img, z_disc],
                            outputs=[valid, fake, validity_interpolated])
        self.critic_model.compile(loss=[self.wasserstein_loss,
                                              self.wasserstein_loss,
                                              partial_gp_loss],
                                        optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Sampled noise for input to generator
        z_gen = Input(shape=(100,))
        # Generate images based of noise
        img = self.generator(z_gen)
        # Discriminator determines validity
        valid = self.critic(img)
        # Defines generator model
        self.generator_model = Model(z_gen, valid)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)


    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)


    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake =  np.ones((batch_size, 1))
        dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                for _ in range(self.n_critic):
                    global_step += 1
                    imgs = X_train[index * batch_size:(index + 1) * batch_size]
                    # Sample generator input
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                    # Train the critic
                    d_loss = self.critic_model.train_on_batch([imgs, noise],
                                                              [valid, fake, dummy])

                    # ---------------------
                    #  Train Generator
                    # ---------------------

                g_loss = self.generator_model.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch%d step%d [D loss: %f] [G loss: %f]" % (epoch, global_step, d_loss[0], g_loss))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch, global_step)

    def sample_images(self, epoch, global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        generated_images = self.generator.predict(noise)

        generated_images = np.asarray((generated_images * 127.5 + 127.5).astype(np.uint8))

        def vis_square(data, padsize=1, padval=0):
            # force the number of filters to be square
            n = int(np.ceil(np.sqrt(data.shape[0])))
            padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
            data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))

            # tile the filters into an image
            data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
            data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
            return data

        img = vis_square(generated_images)
        if not os.path.isdir('images_wpgan_cifar10'):
            os.mkdir('images_wpgan_cifar10')
        Image.fromarray(img).save(
            "images_wpgan_cifar10/epoch_%d_step_%d.png" % (epoch, global_step))


if __name__ == '__main__':
    wgan = WGANGP()
    wgan.train(epochs=10, batch_size=32, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8192)              827392    
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       262272    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 128)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 32, 32, 128)       0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch0 step5 [D loss: 41.168491] [G loss: -0.069504]
epoch0 step10 [D loss: 49.962944] [G loss: -0.136793]
epoch0 step15 [D loss: 44.444565] [G loss: -0.071928]
epoch0 step20 [D loss: 37.074509] [G loss: -0.222208]
epoch0 step25 [D loss: 39.464947] [G loss: -0.109974]
epoch0 step30 [D loss: 30.682512] [G loss: -0.155986]
epoch0 step35 [D loss: 23.939909] [G loss: -0.219287]
epoch0 step40 [D loss: 25.078129] [G loss: -0.033152]
epoch0 step45 [D loss: 18.928408] [G loss: -0.104180]
epoch0 step50 [D loss: 18.227310] [G loss: -0.146719]
epoch0 step55 [D loss: 18.945900] [G loss: -0.206374]
epoch0 step60 [D loss: 13.453578] [G loss: -0.111615]
epoch0 step65 [D loss: 13.019513] [G loss: -0.155081]
epoch0 step70 [D loss: 10.912840] [G loss: -0.089241]
epoch0 step75 [D loss: 11.780775] [G loss: -0.049562]
epoch0 step80 [D loss: 8.191308] [G loss: 0.056165]
epoch0 step85 [D loss: 8.831063] [G loss: 0.048138]
epoch0 step90 [D loss: 8.791668] [G loss: 0.149269]
epoch0 step95 [D loss: 6.804173] [G

epoch0 step775 [D loss: -1.080407] [G loss: 2.180385]
epoch0 step780 [D loss: -0.800211] [G loss: 2.417128]
epoch0 step785 [D loss: -0.933736] [G loss: 2.175747]
epoch0 step790 [D loss: -1.253410] [G loss: 2.617570]
epoch0 step795 [D loss: -0.592680] [G loss: 2.708334]
epoch0 step800 [D loss: -1.330419] [G loss: 3.125541]
epoch0 step805 [D loss: -1.418691] [G loss: 2.875823]
epoch0 step810 [D loss: -1.213320] [G loss: 3.304039]
epoch0 step815 [D loss: -1.496240] [G loss: 3.121229]
epoch0 step820 [D loss: -1.319165] [G loss: 3.229015]
epoch0 step825 [D loss: 0.032796] [G loss: 2.722104]
epoch0 step830 [D loss: -1.064786] [G loss: 3.251639]
epoch0 step835 [D loss: -0.714844] [G loss: 2.630995]
epoch0 step840 [D loss: -0.900309] [G loss: 3.109566]
epoch0 step845 [D loss: -0.438599] [G loss: 2.361563]
epoch0 step850 [D loss: -0.116286] [G loss: 2.341780]
epoch0 step855 [D loss: -0.062834] [G loss: 2.456864]
epoch0 step860 [D loss: -0.804647] [G loss: 2.394067]
epoch0 step865 [D loss: -0.84

...

epoch7 step56175 [D loss: -0.029778] [G loss: 0.447178]
epoch7 step56180 [D loss: -0.417738] [G loss: 0.225886]
epoch7 step56185 [D loss: -0.474649] [G loss: 0.364037]
epoch7 step56190 [D loss: -0.228568] [G loss: 0.058094]
epoch7 step56195 [D loss: 0.140913] [G loss: -0.172285]
epoch7 step56200 [D loss: 0.157134] [G loss: 0.045973]
epoch7 step56205 [D loss: 0.254809] [G loss: 0.029818]
epoch7 step56210 [D loss: 0.295339] [G loss: 0.080631]
epoch7 step56215 [D loss: -0.451621] [G loss: 0.178491]
epoch7 step56220 [D loss: 0.320752] [G loss: -0.220560]
epoch7 step56225 [D loss: -0.100914] [G loss: 0.131026]
epoch7 step56230 [D loss: -0.214192] [G loss: -0.231354]
epoch7 step56235 [D loss: -0.111776] [G loss: -0.119568]
epoch7 step56240 [D loss: -0.031640] [G loss: 0.244960]
epoch7 step56245 [D loss: -0.334097] [G loss: 0.013059]
epoch7 step56250 [D loss: -0.146182] [G loss: 0.139578]
epoch7 step56255 [D loss: -0.498729] [G loss: 0.055952]
epoch7 step56260 [D loss: -0.341920] [G loss: 0.5

epoch7 step56905 [D loss: -0.049113] [G loss: -0.084727]
epoch7 step56910 [D loss: -0.335471] [G loss: 0.282292]
epoch7 step56915 [D loss: 0.007947] [G loss: -0.226416]
epoch7 step56920 [D loss: -0.608994] [G loss: 0.264753]
epoch7 step56925 [D loss: -0.400930] [G loss: -0.226720]
epoch7 step56930 [D loss: -0.382636] [G loss: -0.396063]
epoch7 step56935 [D loss: -0.341425] [G loss: -0.209003]
epoch7 step56940 [D loss: -0.215372] [G loss: -0.121386]
epoch7 step56945 [D loss: -0.739238] [G loss: -0.067079]
epoch7 step56950 [D loss: 0.263323] [G loss: -0.378870]
epoch7 step56955 [D loss: 0.094887] [G loss: -0.105952]
epoch7 step56960 [D loss: 0.242609] [G loss: -0.267968]
epoch7 step56965 [D loss: 0.437557] [G loss: -0.050531]
epoch7 step56970 [D loss: -0.684261] [G loss: 0.093453]
epoch7 step56975 [D loss: 0.110967] [G loss: 0.048364]
epoch7 step56980 [D loss: 0.127261] [G loss: -0.300892]
epoch7 step56985 [D loss: -0.369138] [G loss: 0.140083]
epoch7 step56990 [D loss: -0.041310] [G los

epoch7 step57645 [D loss: -0.038594] [G loss: 0.064708]
epoch7 step57650 [D loss: -0.465069] [G loss: 0.183217]
epoch7 step57655 [D loss: -0.076706] [G loss: 0.149365]
epoch7 step57660 [D loss: -0.663584] [G loss: 0.179644]
epoch7 step57665 [D loss: -0.162148] [G loss: 0.137324]
epoch7 step57670 [D loss: -0.257956] [G loss: -0.296374]
epoch7 step57675 [D loss: -0.305147] [G loss: 0.412524]
epoch7 step57680 [D loss: -0.415096] [G loss: -0.324793]
epoch7 step57685 [D loss: -0.167114] [G loss: -0.170837]
epoch7 step57690 [D loss: 0.001319] [G loss: -0.276689]
epoch7 step57695 [D loss: -0.227791] [G loss: -0.474327]
epoch7 step57700 [D loss: 0.331503] [G loss: -0.205400]
epoch7 step57705 [D loss: 0.251462] [G loss: -0.158433]
epoch7 step57710 [D loss: 0.031832] [G loss: -0.436743]
epoch7 step57715 [D loss: 0.151286] [G loss: -0.299496]
epoch7 step57720 [D loss: 0.481422] [G loss: -0.063660]
epoch7 step57725 [D loss: -0.225622] [G loss: -0.422502]
epoch7 step57730 [D loss: -0.192049] [G los

epoch7 step58375 [D loss: 0.012665] [G loss: -0.366902]
epoch7 step58380 [D loss: 0.338818] [G loss: -0.619819]
epoch7 step58385 [D loss: -0.141539] [G loss: -0.396830]
epoch7 step58390 [D loss: 0.328664] [G loss: -0.248385]
epoch7 step58395 [D loss: 0.241072] [G loss: 0.062780]
epoch7 step58400 [D loss: -0.588533] [G loss: 0.111976]
epoch7 step58405 [D loss: -0.221303] [G loss: -0.333249]
epoch7 step58410 [D loss: 0.087393] [G loss: 0.062245]
epoch7 step58415 [D loss: 0.147882] [G loss: -0.097816]
epoch7 step58420 [D loss: 0.354697] [G loss: -0.071849]
epoch7 step58425 [D loss: -0.259861] [G loss: 0.111004]
epoch7 step58430 [D loss: -0.144928] [G loss: 0.027576]
epoch7 step58435 [D loss: -0.329772] [G loss: 0.356504]
epoch7 step58440 [D loss: -0.452923] [G loss: 0.206843]
epoch7 step58445 [D loss: -0.452924] [G loss: 0.601502]
epoch7 step58450 [D loss: -0.132129] [G loss: -0.027818]
epoch7 step58455 [D loss: -0.547387] [G loss: 0.385756]
epoch7 step58460 [D loss: 0.046321] [G loss: -0

KeyboardInterrupt: 